In [42]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
## import warnings
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=FutureWarning, module='sklearn')
warnings.filterwarnings(action='ignore', category=DataConversionWarning, module='sklearn')

In [2]:
df = pd.read_csv('dataR2.csv')
df.head()

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114,1
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786,1
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697,1
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220,1
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920,1


In [38]:
X = df.drop('Classification',axis=1)
y = df['Classification']

In [39]:
sc_X = StandardScaler()
X = sc_X.fit_transform(X)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [43]:
# Definição dos parâmetros a serem testados
tree_params = {'criterion': ['gini', 'entropy'], 'max_depth': range(10, 101, 10)}
knn_params = {'n_neighbors': [1, 3, 5, 11, 21, 31]}

# Divisão dos dados em k-folds
n_splits = 5
skf_outer = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Decision Tree
tree_model = DecisionTreeClassifier(random_state=42)
tree_grid = GridSearchCV(tree_model, tree_params, cv=n_splits)
tree_grid.fit(X, y)

# K-vizinhos
knn_model = KNeighborsClassifier()
knn_grid = GridSearchCV(knn_model, knn_params, cv=n_splits)
knn_grid.fit(X, y)

# Melhor modelo e conjunto de hiperparâmetros
if tree_grid.best_score_ > knn_grid.best_score_:
    best_model = tree_grid.best_estimator_
    best_params = tree_grid.best_params_
    best_score = tree_grid.best_score_
else:
    best_model = knn_grid.best_estimator_
    best_params = knn_grid.best_params_
    best_score = knn_grid.best_score_

# Retreinamento do modelo com o melhor conjunto de hiperparâmetros usando todos os dados
best_model.set_params(**best_params)
best_model.fit(X, y)

# Avaliação do desempenho com dados de teste (substitua pelos seus próprios dados)
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Impressão da acurácia média e melhores hiperparâmetros
print("Acurácia média:", best_score)
print("Melhores hiperparâmetros:", best_params)
print("Acurácia no conjunto de teste:", accuracy)


Acurácia média: 0.7510869565217392
Melhores hiperparâmetros: {'criterion': 'entropy', 'max_depth': 10}
Acurácia no conjunto de teste: 1.0
